#Preparing the dataset

In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load data
df = pd.read_csv("osc_sheet_11_cleaned_1000.csv")

print("Shape:", df.shape)
print("\nData Types:\n", df.dtypes)
print("\nMissing Values:\n", df.isnull().sum())

# Encode first four columns (includes industry)
id_cols = df.columns[:4]

for col in id_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

df = df.rename(columns={
    "Unnamed: 0": "company_id",
    "Unnamed: 1": "industry_id",
    "Unnamed: 2": "isin_code",
    "Unnamed: 3": "nse_symbol"
})
id_cols = ["company_id", "industry_id", "isin_code","nse_symbol"]
# Rename financial columns
df = df.rename(columns={
    "CMIE Expr": "revenue",
    "CMIE Expr.2": "ebit",
    "CMIE Expr.3": "net_profit",
    "CMIE Expr.4": "total_assets",
    "CMIE Expr.5": "equity",
    "CMIE Expr.6": "total_debt",
    "CMIE Expr.7": "current_assets",
    "CMIE Expr.8": "current_liabilities",
    "CMIE Expr.9": "interest_expense",
    "CMIE Expr.10": "inventory",
    "CMIE Expr.11": "cogs",
    "CMIE Expr.12": "operating_cash_flow",
    "CMIE Expr.13": "capital_employed",
    "CMIE Expr.14": "eps",
    "CMIE Expr.15": "book_value_per_share",
    "CMIE Expr.16": "share_price",
    "CMIE Expr.17": "year"
})

financial_cols = [
    "revenue", "ebit", "net_profit", "total_assets", "equity",
    "total_debt", "current_assets", "current_liabilities",
    "interest_expense", "inventory", "cogs",
    "operating_cash_flow", "capital_employed",
    "eps", "book_value_per_share", "share_price"
]

# Clean financial columns
for col in financial_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", "", regex=False)
        .replace(["NA", "-", "nan", "None", ""], np.nan)
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")

df[financial_cols] = df[financial_cols].fillna(df[financial_cols].median())

# Save cleaned data WITH encoded columns
df.to_csv("cleaned_financial_data.csv", index=False)

# =========================
# RATIO CALCULATIONS
# =========================

ratios = df[id_cols + ["year"]].copy()

ratios["ROE"] = df["net_profit"] / df["equity"]
ratios["ROCE"] = df["ebit"] / df["capital_employed"]
ratios["Net Profit Margin"] = df["net_profit"] / df["revenue"]
ratios["Asset Turnover"] = df["revenue"] / df["total_assets"]
ratios["Current Ratio"] = df["current_assets"] / df["current_liabilities"]
ratios["Debt to Equity"] = df["total_debt"] / df["equity"]
ratios["Interest Coverage Ratio"] = df["ebit"] / df["interest_expense"]
ratios["Operating Cash Flow to Net Profit"] = (
    df["operating_cash_flow"] / df["net_profit"]
)
ratios["DOS"] = (df["inventory"] / df["cogs"]) * 365
ratios["PE"] = df["share_price"] / df["eps"]
ratios["PB"] = df["share_price"] / df["book_value_per_share"]

# CAGR calculation
df_sorted = df.sort_values("year")

def calculate_cagr(series):
    n = len(series) - 1
    if n <= 0 or series.iloc[0] == 0:
        return np.nan
    return (series.iloc[-1] / series.iloc[0]) ** (1 / n) - 1

ratios["Revenue CAGR"] = calculate_cagr(df_sorted["revenue"])
ratios["Earnings CAGR"] = calculate_cagr(df_sorted["net_profit"])

# Save ratios WITH encoded columns
ratios.to_csv("financial_ratios.csv", index=False)

print("✅ Cleaned data & ratios files created successfully")


Shape: (1004, 31)

Data Types:
 Unnamed: 0      object
Unnamed: 1      object
Unnamed: 2      object
Unnamed: 3      object
CMIE Expr       object
CMIE Expr.1     object
CMIE Expr.2     object
CMIE Expr.3     object
CMIE Expr.4     object
CMIE Expr.5     object
CMIE Expr.6     object
CMIE Expr.7     object
CMIE Expr.8     object
CMIE Expr.9     object
CMIE Expr.10    object
CMIE Expr.11    object
CMIE Expr.12    object
CMIE Expr.13    object
CMIE Expr.14    object
CMIE Expr.15    object
CMIE Expr.16    object
CMIE Expr.17    object
CMIE Expr.18    object
CMIE Expr.19    object
CMIE Expr.20    object
CMIE Expr.21    object
CMIE Expr.22    object
CMIE Expr.23    object
CMIE Expr.24    object
CMIE Expr.25    object
CMIE Expr.26    object
dtype: object

Missing Values:
 Unnamed: 0        4
Unnamed: 1        4
Unnamed: 2        4
Unnamed: 3       17
CMIE Expr         6
CMIE Expr.1       6
CMIE Expr.2       9
CMIE Expr.3     308
CMIE Expr.4       6
CMIE Expr.5     302
CMIE Expr.6      48
CMI

#Dataset with the target variable included

In [21]:
import pandas as pd
import numpy as np

df = pd.read_csv("financial_ratios.csv")
def classify_investable(row):
    score = 0

    # Profitability
    if row["ROE"] >= 0.15:
        score += 1
    if row["ROCE"] >= 0.12:
        score += 1
    if row["Net Profit Margin"] >= 0.10:
        score += 1

    # Efficiency & Risk
    if row["Asset Turnover"] >= 0.5:
        score += 1
    if row["Debt to Equity"] <= 1.0:
        score += 1
    if row["Interest Coverage Ratio"] >= 3:
        score += 1

    # Liquidity (optional but strong signal)
    if row["Current Ratio"] >= 1.5:
        score += 1

    # Final decision
    return 1 if score >= 4 else 0


df["investable"] = df.apply(classify_investable, axis=1)

print(df["investable"].value_counts())
df.to_csv("financial_ratios_with_target.csv", index=False)


investable
0    737
1    267
Name: count, dtype: int64


In [22]:
import numpy as np

# Replace +inf and -inf with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# (Optional but recommended) check
print("Infinity check:", np.isinf(df.select_dtypes(include=[np.number])).sum().sum())
print("NaN count:\n", df.isna().sum())


Infinity check: 0
NaN count:
 company_id                              0
industry_id                             0
isin_code                               0
nse_symbol                              0
year                                    0
ROE                                     0
ROCE                                    0
Net Profit Margin                    1004
Asset Turnover                       1004
Current Ratio                           2
Debt to Equity                          0
Interest Coverage Ratio                 0
Operating Cash Flow to Net Profit       0
DOS                                     0
PE                                      0
PB                                      4
Revenue CAGR                         1004
Earnings CAGR                           0
investable                              0
dtype: int64


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("financial_ratios_with_target.csv")

# Extract 4-digit year safely
df["year"] = (
    df["year"]
    .astype(str)
    .str.extract(r"(\d{4})")
    .astype(float)
)

# Fill missing years
df["year"] = df["year"].fillna(df["year"].median())

# Define X and y
y = df["investable"]

X = df.drop(columns=[
    "investable",
    "isin_code",
    "nse_symbol"
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)
# 1. Check no overlap between train and test
train_indices = set(X_train.index)
test_indices = set(X_test.index)

print("Overlap between train & test:",
      len(train_indices.intersection(test_indices)))


# 2. Verify class distribution preservation
print("\nTarget distribution (original):")
print(y.value_counts(normalize=True))

print("\nTarget distribution (train):")
print(y_train.value_counts(normalize=True))

print("\nTarget distribution (test):")
print(y_test.value_counts(normalize=True))


# 3. Check shapes consistency
print("\nFeature count check:")
print("X_train features:", X_train.shape[1])
print("X_test features:", X_test.shape[1])



Train size: (803, 16)
Test size: (201, 16)
Overlap between train & test: 0

Target distribution (original):
investable
0    0.734064
1    0.265936
Name: proportion, dtype: float64

Target distribution (train):
investable
0    0.733499
1    0.266501
Name: proportion, dtype: float64

Target distribution (test):
investable
0    0.736318
1    0.263682
Name: proportion, dtype: float64

Feature count check:
X_train features: 16
X_test features: 16


In [9]:
drop_cols = ["Net Profit Margin", "Asset Turnover", "Revenue CAGR"]

X_train = X_train.drop(columns=drop_cols)
X_test  = X_test.drop(columns=drop_cols)

In [10]:
import numpy as np

# Replace +inf and -inf with NaN
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_test  = X_test.replace([np.inf, -np.inf], np.nan)


In [11]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(
        max_iter=1000,
        class_weight="balanced",
        random_state=42
    ))
])


In [12]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

skf = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

cv_scores = cross_val_score(
    pipeline,
    X_train,
    y_train,
    cv=skf,
    scoring="roc_auc"
)

print("CV ROC-AUC scores:", cv_scores)
print("Mean CV ROC-AUC:", cv_scores.mean())


CV ROC-AUC scores: [0.81907765 0.79325976 0.7763106  0.8157424  0.81315577]
Mean CV ROC-AUC: 0.8035092354761255
